In [8]:
!pip install gsutil

  Using cached monotonic-1.6-py2.py3-none-any.whl (8.2 kB)
  Created wheel for gsutil: filename=gsutil-5.27-py3-none-any.whl size=3785301 sha256=2117686b8f95971d7e4843bc5160b45f14672f9b86eaeac0c9e07638aa56dba1
  Stored in directory: c:\users\renal\appdata\local\pip\cache\wheels\de\7e\15\d4fc45223984db92cbe31d7cd0d85c30bc2a0e0fc8753764fd
  Created wheel for crcmod: filename=crcmod-1.7-cp39-cp39-win_amd64.whl size=25135 sha256=7999638b546ee19c709d126f048321817ef819d21d5f3dbd4cf1630af2063436
  Stored in directory: c:\users\renal\appdata\local\pip\cache\wheels\4a\6c\a6\ffdd136310039bf226f2707a9a8e6857be7d70a3fc061f6b36
  Created wheel for gcs-oauth2-boto-plugin: filename=gcs_oauth2_boto_plugin-3.0-py3-none-any.whl size=23220 sha256=d12dcc5b37f5a474369e444795265d7d4383dcbe937a153805e9734b0a80779b
  Stored in directory: c:\users\renal\appdata\local\pip\cache\wheels\f1\40\ff\39d839bc4fba3fe04886bba7931d11813caf5a0b485c372714
  Created wheel for retry-decorator: filename=retry_decorator-1.1.1-

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
import google.generativeai as genai
genai.configure(api_key=os.getenv('GCP_API_KEY'))

In [4]:
model = genai.GenerativeModel("gemini-pro-vision")

In [5]:
from IPython.display import Markdown, display
from vertexai.preview.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Image,
    Part,
)

### Download custom Python modules and utilities

In [6]:
import os
import urllib.request
import sys

if not os.path.exists("utils"):
    os.makedirs("utils")

    
# download the helper scripts from utils folder
url_prefix = "https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/retrieval-augmented-generation/utils/"
files = ["intro_multimodal_rag_utils.py"]

for fname in files:
    urllib.request.urlretrieve(f"{url_prefix}/{fname}", filename=f"utils/{fname}")

### Get documents and images from GCS

In [9]:
# download documents and images used in this notebook
!gsutil -m rsync -r gs://github-repo/rag/intro_multimodal_rag .
print("Synchronization completed")

Synchronization completed


Building synchronization state...
Starting synchronization...
Copying gs://github-repo/rag/intro_multimodal_rag/text_query_answer_02.png...
/ [0/5 files][    0.0 B/823.1 KiB]   0% Done                                    
Copying gs://github-repo/rag/intro_multimodal_rag/google-10k-sample-14pages.pdf...
/ [0/5 files][    0.0 B/823.1 KiB]   0% Done                                    
Copying gs://github-repo/rag/intro_multimodal_rag/class_a_share.png...
/ [0/5 files][    0.0 B/823.1 KiB]   0% Done                                    
Copying gs://github-repo/rag/intro_multimodal_rag/tac_table_revenue.png...
/ [0/5 files][    0.0 B/823.1 KiB]   0% Done                                    
-
- [1/5 files][ 26.4 KiB/823.1 KiB]   3% Done                                    
- [2/5 files][ 95.0 KiB/823.1 KiB]  11% Done                                    
Copying gs://github-repo/rag/intro_multimodal_rag/text_query_answer_01.png...
- [2/5 files][ 95.0 KiB/823.1 KiB]  11% Done                     

### Extract and store metadata of text and images from a document

In [14]:
!pip install pymupdf

In [15]:
from utils.intro_multimodal_rag_utils import get_document_metadata

In [20]:
# Specify the PDF path
pdf_path = "google-10k-sample-14pages.pdf"

# Specify the image description prompt. Change it
image_description_prompt = """Explain what is going on in the image.
If it's a table, extract all elements of the table. 
If it's a graph, explain the findings in the graph.
Do not include any numbers that are not mentioned in the image:"""

# Extract text and image metadata from the PDF document
text_metadata_df, image_metadata_df = get_document_metadata(
    "gemini-test-project-408107",  # Assuming this is a positional argument
    model,                         # Assuming this is a positional argument
    pdf_path=pdf_path,
    image_save_dir="images",
    image_description_prompt=image_description_prompt,
    embedding_size=1408,
    text_emb_text_limit=1000
)


print("--- Completed processing. ---")

Processing page: 1


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [21]:
text_metadata_df.head()

NameError: name 'text_metadata_df' is not defined

In [ ]:
image_metadata_df.head()

### Import the helper functions to implement RAG¶

In [ ]:
from utils.intro_multimodal_rag_utils import (
    get_similar_text_from_query,
    print_text_to_text_citation,
    get_similar_image_from_query,
    print_text_to_image_citation,
    get_gemini_response,
    display_images,
)


### Search similar text with text query

In [ ]:
query = "I need details for basic and diluted net income per share of Class A, Class B, and Class C share for google?"

In [ ]:
# Matching user text query with "chunk_embedding" to find relevant chunks.
matching_results_text = get_similar_text_from_query(
    PROJECT_ID,
    query,
    text_metadata_df,
    column_name="text_embedding_chunk",
    top_n=3,
    embedding_size=1408,
    chunk_text=True,
)

# Print the matched text citations
print_text_to_text_citation(matching_results_text, print_top=True, chunk_text=True)


### Search similar images with text query

In [ ]:
matching_results_image = get_similar_image_from_query(
    PROJECT_ID,
    text_metadata_df,
    image_metadata_df,
    query=query,
    column_name="text_embedding_from_image_description",  # Use image description text embedding
    image_emb=False,  # Use text embedding instead of image embedding
    top_n=3,
    embedding_size=1408,
)

# Markdown(print_text_to_image_citation(matching_results_image, print_top=True))
print("\n **** Result: ***** \n")

# Display the top matching image
display(matching_results_image[0]["image_object"])

In [ ]:
## you can check the citations to probe further.
## check the "image description:" which is a description extracted through gemini which helped search our query.
Markdown(print_text_to_image_citation(matching_results_image, print_top=True))

## Image Search

### Search similar image with image query

In [1]:
# You can find a similar image as per the images you have in the metadata.
# In this case, you have a table (picked from the same document source) and you would like to find similar tables in the document.
image_query_path = "tac_table_revenue.png"

# Print a message indicating the input image
print("***Input image from user:***")

# Display the input image
Image.load_from_file(image_query_path)


***Input image from user:***


NameError: name 'Image' is not defined

In [ ]:
# Search for Similar Images Based on Input Image and Image Embedding

matching_results_image = get_similar_image_from_query(
    PROJECT_ID,
    text_metadata_df,
    image_metadata_df,
    query=query,  # Use query text for additional filtering (optional)
    column_name="mm_embedding_from_img_only",  # Use image embedding for similarity calculation
    image_emb=True,
    image_query_path=image_query_path,  # Use input image for similarity calculation
    top_n=3,  # Retrieve top 3 matching images
    embedding_size=1408,  # Use embedding size of 1408
)

print("\n **** Result: ***** \n")

# Display the Top Matching Image
display(
    matching_results_image[0]["image_object"]
)  # Display the top matching image object (Pillow Image)


In [ ]:
# Display citation details for the top matching image
print_text_to_image_citation(matching_results_image, print_top=True)

In [ ]:
# Check Other Matched Images (Optional)
# You can access the other two matched images using:

print("---------------Matched Images------------------\n")
display_images(
    [
        matching_results_image[0]["img_path"],
        matching_results_image[1]["img_path"],
    ],
    resize_ratio = 0.8
)

In [ ]:
image_query_path = "class_a_share.png"

# Print a message indicating the input image
print("***Input image from user:***")

# Display the input image
Image.load_from_file(image_query_path)

In [ ]:
# Load the input image using Pillow
user_image_object = Image.load_from_file(image_query_path)

# Define the comparison query
compare_query = """Question: How has nasdaq performed with respect to Class A and Class B shares of Google?
Answer: """
instructions = """instructions: Compare two images and base your reasoning only on the images provided.
Provide detail reasoning of your conclusions.
Images: """

# Find similar images based on the input image
image_selected_based_on_source_image = get_similar_image_from_query(
    PROJECT_ID,
    text_metadata_df,
    image_metadata_df,
    image_query_path=image_query_path,
    column_name="mm_embedding_from_img_only",
    image_emb=True,
    top_n=3,
    embedding_size=1408,
)

# Select the best matching image from the search results
selected_image_object = image_selected_based_on_source_image[0]["image_object"]

# Prepare the model input
model_input = [instructions, user_image_object, selected_image_object, compare_query]

# Generate Gemini response with streaming output
Markdown(get_gemini_response(model, model_input=model_input, stream=True))


In [ ]:
# image selected by the model to make the comparision based on user query
Image.load_from_file(image_selected_based_on_source_image[0]["img_path"])


In [ ]:
# citations
print_text_to_image_citation(image_selected_based_on_source_image, print_top=True)

## Multimodal retrieval augmented generation (RAG)

In [ ]:
# this time we are not passing any images, but just a simple text query.

query = """Question: How has nasdaq and s&p performed with respect to class A shares and class C shares?
Which one would be better to buy and why?
Answer: """

# query = """Question: Find the total revenues and other related financial numbers for Alphabet
# Answer: """

In [ ]:
# Retrieve relevant chunks of text based on the query
matching_results_chunks_data = get_similar_text_from_query(
    PROJECT_ID,
    query,
    text_metadata_df,
    column_name="text_embedding_chunk",
    top_n=5,
    embedding_size=1408,
    chunk_text=True,
)


In [ ]:
# Get all relevant images based on user query
matching_results_image_fromdescription_data = get_similar_image_from_query(
    PROJECT_ID,
    text_metadata_df,
    image_metadata_df,
    query=query,
    column_name="text_embedding_from_image_description",
    image_emb=False,
    top_n=3,
    embedding_size=1408,
)


In [ ]:
# combine all the selected relevant text chunks
context_text = []
for key, value in matching_results_chunks_data.items():
    context_text.append(value["chunk_text"])
final_context_text = "\n".join(context_text)

# combine all the relevant images and their description generated by Gemini
context_images = []
for key, value in matching_results_image_fromdescription_data.items():
    context_images.extend(
        ["Image: ", value["image_object"], "Caption: ", value["image_description"]]
    )


In [ ]:
instructions = """The context of extraction of destails should be based on the text context given in "text_context" and Image context given in "image_context" along with its Caption: \n
Base your response on "text_context" and "image_context". Do not use any numbers or percentages that are not present in the "image_context".
Do not include any cumulative total return in the answer. Context: 
"""

final_prompt = [
    query,
    instructions,
    "text_context:",
    "\n".join(context_text),
    "image_context:",
]
final_prompt.extend(context_images)


In [ ]:
Markdown(get_gemini_response(model, model_input = final_prompt,stream=True))

In [ ]:
print("---------------Matched Images------------------\n")
display_images(
    [
        matching_results_image_fromdescription_data[0]["img_path"],
        matching_results_image_fromdescription_data[1]["img_path"],
    ],
    resize_ratio = 0.8
)


In [ ]:
# Image citations. You can check how Gemini generated metadata helped in grounding the answer.

print_text_to_image_citation(matching_results_image_fromdescription_data, print_top=False)

In [ ]:
# Text citations

print_text_to_text_citation(
    matching_results_chunks_data,
    print_top=False,
    chunk_text=True,
)
